In [1]:
import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random
import pickle

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
class NormalDerivativeVisualizer:
    def __init__(self, pcd, radius=2.0):
        self.pcd = pcd
        self.points = np.asarray(pcd.points)
        self.pcd.paint_uniform_color([0.5, 0.5, 0.5])
        self.kdtree = cKDTree(self.points)
        self.radius = radius
        self.vis = o3d.visualization.VisualizerWithKeyCallback()
        self.vis.create_window("Normal Derivative Visualizer")
        self.vis.register_key_callback(262, self.compute_and_visualize)  # Right arrow key
        self.vis.add_geometry(self.pcd)
        print("Press → (Right Arrow) to sample a point and compute normal derivatives.")

    def compute_normal_derivative(self, neighbors, normals):
        """
        Estimate the normal gradient direction using finite differences.

        Args:
            neighbors (np.ndarray): The 3D coordinates of neighborhood points (Nx3).
            normals (np.ndarray): The normal vectors at those points (Nx3).

        Returns:
            np.ndarray: A single 3D vector representing the direction of normal change.
        """
        normal_diff = normals - normals.mean(axis=0)  # Compute deviation from mean normal
        distances = np.linalg.norm(neighbors - neighbors.mean(axis=0), axis=1, keepdims=True)  # Point spread

        valid = distances.squeeze() > 1e-6  # Avoid division by zero

        if np.any(valid):
            # Normalize difference by distance (finite difference approximation)
            gradient_direction = np.mean(normal_diff[valid] / distances[valid], axis=0)
            return gradient_direction / np.linalg.norm(gradient_direction)  # Normalize
        return np.zeros(3)
    
    def compute_normal_derivative_pca(self, normals):
        """Compute dominant direction of normal variation using PCA."""
        cov = np.cov(normals.T)
        eig_vals, eig_vecs = np.linalg.eigh(cov)
        return eig_vecs[:, np.argmax(eig_vals)]  # Largest eigenvector = main change direction


    def align_normals(self, reference_normal, neighbor_directions):
        aligned_normals = np.array(neighbor_directions)
        
        # Check dot product: If negative, flip the normal
        for i in range(len(aligned_normals)):
            if np.dot(reference_normal, aligned_normals[i]) < 0:
                aligned_normals[i] = -aligned_normals[i]

        return aligned_normals
    
    def compute_and_visualize(self, vis):
        """Sample a point, compute its neighborhood normals, and visualize derivatives."""
        sample_idx = random.randint(0, len(self.points) - 1)
        sample_point = self.points[sample_idx]

        # Find neighbors
        neighbor_indices = self.kdtree.query_ball_point(sample_point, self.radius)
        neighbor_indices.append(sample_idx)
        neighbors = self.points[neighbor_indices]

        if len(neighbors) < 3:
            print("Not enough neighbors, skipping...")
            return

        # Compute normals
        normals = np.array(util.calculateNormals(neighbor_indices, self.points, radius=2))
        aligned_normals = self.align_normals(normals[0], normals)
        # Compute normal gradient
        normal_gradient = self.compute_normal_derivative_pca(aligned_normals)

        if hasattr(self, "normal_lines"):
            self.vis.remove_geometry(self.normal_lines)
        if hasattr(self, "gradient_lines"):
            self.vis.remove_geometry(self.gradient_lines)

        # Visualize
        self.normal_lines = self.create_lines(neighbors, aligned_normals, color=(0, 0, 1))  # Blue = Normals
        self.gradient_lines = self.create_lines([sample_point], [normal_gradient], color=(1, 0, 0), scale=2)  # Red = Gradient

        #self.vis.clear_geometries()
        #self.vis.add_geometry(self.pcd)
        self.vis.add_geometry(self.normal_lines)
        self.vis.add_geometry(self.gradient_lines)

        view_ctl = self.vis.get_view_control()
        lookat = sample_point
        zoom = 0.080000000000000002
        front = [-0.024106890455448116,-0.57254772319971181,0.81951690799604338]
        up =  [0.014828165865396817,0.81946017828866602,0.57294427451208185]
        view_ctl.set_lookat(lookat)  # Set the point the camera is looking at
        view_ctl.set_up(up)      # Set the up direction of the camera
        view_ctl.set_front(front)  # Set the front direction of the camera
        view_ctl.set_zoom(zoom)          # Set the zoom factor of the camera

        #print(f"Sampled Point Index: {sample_idx}, Normals: {normals}, Gradient: {normal_gradients}")

    def create_lines(self, points, vectors, color=(1, 0, 0), scale=1.0):
        """Creates Open3D line geometry for visualization."""
        start_points = np.array(points)
        end_points = start_points + scale * np.array(vectors)
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(np.vstack((start_points, end_points)))
        line_indices = [[i, i + len(points)] for i in range(len(points))]
        line_set.lines = o3d.utility.Vector2iVector(line_indices)
        line_set.colors = o3d.utility.Vector3dVector([color] * len(line_indices))
        return line_set

    def run(self):
        self.vis.run()
        self.vis.destroy_window()

# Load point cloud
pcd = o3d.io.read_point_cloud("/home/chris/Code/PointClouds/data/ply/CircularVentilationGrateExtraCleanedFull.ply")
visualizer = NormalDerivativeVisualizer(pcd, radius=2.0)
visualizer.run()


Press → (Right Arrow) to sample a point and compute normal derivatives.
